In [31]:

import torch
import torch.nn as nn
import os
import numpy as np
from torch.nn.utils import clip_grad_norm_
from data_utils import Dictionary, Corpus
from torchnlp.datasets import penn_treebank_dataset
from GWDC import GWDC

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#dataset
ROOT = './data/penn-treebank/'
if not os.path.exists(ROOT):
    penn_treebank_dataset()
# Hyper-parameters
embed_size = 128
hidden_size = 1024
num_layers = 3
num_epochs = 5
num_samples = 1000     # number of words to be sampled
batch_size = 20
seq_length = 30
learning_rate = 0.002

# Load "Penn Treebank" dataset
corpus = Corpus()
ids = corpus.get_data(ROOT+'ptb.train.txt', batch_size)
vocab_size = len(corpus.dictionary)
num_batches = ids.size(1) // seq_length


# RNN based language model
class RNNLM(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(RNNLM, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.GRU(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, x, h):
        # Embed word ids to vectors
        x = self.embed(x)
        
        # Forward propagate LSTM
        out, h = self.lstm(x)
        
        # Reshape output to (batch_size*sequence_length, hidden_size)
        out = out.reshape(out.size(0)*out.size(1), out.size(2))
        
        # Decode hidden states of all time steps
        out = self.linear(out)
        return out, h

model = RNNLM(vocab_size, embed_size, hidden_size, num_layers).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#optimizer = GWDC(model.parameters())
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate,momentum=0.9)
# Truncated backpropagation
def detach(states):
    return [state.detach() for state in states] 

# Train the model
print('Start train!')
for epoch in range(num_epochs):
    # Set initial hidden and cell states
    states = (torch.zeros(num_layers, batch_size, hidden_size).to(device),
              torch.zeros(num_layers, batch_size, hidden_size).to(device))
    
    for i in range(0, ids.size(1) - seq_length, seq_length):
        # Get mini-batch inputs and targets
        inputs = ids[:, i:i+seq_length].to(device)
        targets = ids[:, (i+1):(i+1)+seq_length].to(device)
        
        # Forward pass
        states = detach(states)
        outputs, states = model(inputs, states)
        loss = criterion(outputs, targets.reshape(-1))
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        step = (i+1) // seq_length
        #if step % 100 == 0:
    print ('Epoch [{}/{}], Loss: {:.4f}, Perplexity: {:5.2f}'
                   .format(epoch+1, num_epochs, loss.item(), np.exp(loss.item())))


start train
Epoch [1/5], Loss: 6.9299, Perplexity: 1022.42
Epoch [2/5], Loss: 6.8104, Perplexity: 907.20
Epoch [3/5], Loss: 6.7626, Perplexity: 864.92
Epoch [4/5], Loss: 6.7320, Perplexity: 838.82
Epoch [5/5], Loss: 6.7109, Perplexity: 821.31
